In [1]:
from rag_core import DocumentProcessor, VectorStoreManager, build_retrieval_tool, build_agent
# Step 1: Process documents
doc_processor = DocumentProcessor(folder="folder",
    chunk_size=1000,
    chunk_overlap=100
)


c:\Users\HP\Desktop\mywork\NOTEBOOKS\working_with_llms\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
docs = doc_processor.load_documents()
chunks = doc_processor.chunk_documents(docs)

   Loading: Analysis-of-the-Nigerian-Tax-Reform-Bills.pdf ... Success! (14 pages)
   Loading: CLARIFICATION_ON_THE_IMPLEMENTATION_OF_THE_VALUE_ADDED_TAX_ACT.pdf ... Success! (14 pages)
   Loading: JOINT-REVENUE-BOARD-OF-NIGERIA-(ESTABLISHMENT)-ACT-2025.pdf ... Success! (37 pages)
   Loading: NASS-Journal_Nigeria-Tax-Bill.pdf ... Success! (213 pages)
   Loading: NIGERIA-REVENUE-SERVICE-(ESTABLISHMENT)-ACT-2025.pdf ... Success! (32 pages)
   Loading: Nigeria-Tax-Act-2025.pdf ... Success! (215 pages)
   Loading: NIGERIA-TAX-ADMINISTRATION-ACT-2025.pdf ... Success! (92 pages)
   Loading: State government citations on Tax reform bill.pdf ... Success! (18 pages)
   Loading: The-Nigerian-Tax-Reform-Bills-You-Ask-We-Answer.pdf ... Success! (17 pages)

✅ Total pages loaded: 652 across 9 documents
   Document 1/652: Analysis-of-the-Nigerian-Tax-Reform-Bills.pdf → 2 chunks created
   Document 2/652: Analysis-of-the-Nigerian-Tax-Reform-Bills.pdf → 4 chunks created
   Document 3/652: Analysis-of-th

In [3]:
# Step 2: Create vector store
vectorstore_manager = VectorStoreManager()
vectorstore_manager.create_vectorstore(chunks)


✅ SUCCESS! Vector database built and saved to ./chroma_db
Ready with 1762 searchable chunks from your Tax Reform Bills documents!
RAG system is now ready for accurate answers from the documents!


🔎 Running vector store sanity check...
✅ Sanity check PASSED: Retrieved 3 documents

Result 1:
   Source: Analysis-of-the-Nigerian-Tax-Reform-Bills.pdf
   Page: 5
   Preview: 5Analysis of the Nigerian Tax Reform Bills
c. Increase in V AT Rate
A notable concern regarding the proposed V AT reform is the claim that increasing the V AT rate from 
the current 7.5% to 10%, and u...

Result 2:
   Source: The-Nigerian-Tax-Reform-Bills-You-Ask-We-Answer.pdf
   Page: 7
   Preview: (iii)   - the current formula for sharing VAT among states is based on 20% Basis of distribution
derivation, 50% equality and 30% population. The tax reform proposes a diﬀerent model of 
derivation wh...

Result 3:
   Source: NASS-Journal_Nigeria-Tax-Bill.pdf
   Page: 98
   Preview: Subject  to  the  provisions  of  part  IV

In [6]:
# Step 2: Load vector store
vectorstore_manager = VectorStoreManager()
vectorestore = vectorstore_manager.load_vectorstore()

Loading existing vector store...
✅ Vector store loaded!

🔎 Running vector store sanity check...
✅ Sanity check PASSED: Retrieved 3 documents

Result 1:
   Source: Analysis-of-the-Nigerian-Tax-Reform-Bills.pdf
   Page: 5
   Preview: 5Analysis of the Nigerian Tax Reform Bills
c. Increase in V AT Rate
A notable concern regarding the proposed V AT reform is the claim that increasing the V AT rate from 
the current 7.5% to 10%, and u...

Result 2:
   Source: The-Nigerian-Tax-Reform-Bills-You-Ask-We-Answer.pdf
   Page: 7
   Preview: (iii)   - the current formula for sharing VAT among states is based on 20% Basis of distribution
derivation, 50% equality and 30% population. The tax reform proposes a diﬀerent model of 
derivation wh...

Result 3:
   Source: NASS-Journal_Nigeria-Tax-Bill.pdf
   Page: 98
   Preview: Subject  to  the  provisions  of  part  IV  of  chapter  eight  of  this  Act,  VAT  shall 
 be charged on the value of all taxable supplies at the following rates — 
 a.  taxable  su

In [8]:
from rag_core import build_retrieval_tool

# Test tool directly
retrieve_documents = build_retrieval_tool(vectorstore=vectorestore)
test_result = retrieve_documents.invoke({"query": "Under the Nigeria Tax Act 2025, what is the annual gross income threshold for individuals to be exempt from paying personal income tax, and what is the maximum personal income tax rate for the highest income bracket?"})
print(f"Tool result (first 300 chars):\n{test_result[:300]}...")

Tool result (first 300 chars):
Document 1:
Nigeria Tax Act, 20252025 No. 7A  538
Fourth Schedule
 Section 58
INDIVIDUALS' INCOME TAX RATES
After the relief allowance and exemptions had been granted in accordance
with section 30 (1) of this Act, the taxable income ascertained shall be taxed at
the following rates –
(a) First N800,...


In [13]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
text = "Bonjour tout le monde"
translated = llm.invoke(f"Translate the following text to English:\n{text}").content
print(translated)


Hello everyone.


In [14]:
query = "Tell me about VAT"
refined = llm.invoke(f"Rewrite this query for document retrieval:\n{query}").content
print(refined)


Can you provide information on Value Added Tax (VAT)?


In [17]:
from rag_core import build_retrieval_tool

retrieve_documents = build_retrieval_tool(vectorstore=vectorestore)

query = "What is the VAT rate?"
result = retrieve_documents.invoke({"query": query})
print(result[:1000])


Document 1:
ii. Invoice relating to that part is issued; or 
iii. When payment for that part is received.    
This treatment also applies in the case of instalment, progressive, or periodic payment 
of rent, or any construction, improvement, repair, assembly, manufacturing or similar 
activities or works.   
Where goods are supplied under an instalment credit agreement, the time of supply is 
the earlier of when the goods are delivered and when payment is received by the supplier.  
  
5.0 Rate of Tax     
VAT is chargeable at 7.5% of the value of the goods and services.

---

Document 2:
trailers. 
 
As such, VAT is chargeable on the supply of all items or structures which are not 
included in the definition of building.  Consequently, all persons or  companies 
trading, letting or supplying services with such items or structures are required 
to charge VAT at the rate of 7.5% with effect from 1st September, 2023. 
 
 
 
 
12.2 Penalty Regime    
The following are the new penalty prov

In [19]:
from rag_core import build_agent

test = build_agent(vectorstore=vectorestore)
query_agent = test["query_agent"]
detect_language = test["detect_language"]
needs_retrieval = test["needs_retrieval"]
rewrite_query = test["rewrite_query"]
extract_sources = test["extract_sources"]
retrieval_tool = test["retrieval_tool"]

# Test a query
response = query_agent("Wetin be the VAT rate according to the 2025 Tax Reform Bill?")
import json
print(json.dumps(response, indent=2))


NameError: name 'Dict' is not defined

In [ ]:
import requests

# Query endpoint
response = requests.post(
    "http://localhost:8000/query",
    json={"question": "What is machine learning?"}
)

result = response.json()
print(f"Q: {result['question']}")
print(f"A: {result['answer']}")